In [2]:
!pip install --upgrade geonamescache


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 32.0 MB/s eta 0:00:0000:010:01


In [3]:
import geonamescache
import pandas as pd


In [4]:
# Initialize cache and pull cities dict
gc = geonamescache.GeonamesCache()
cities_dict = gc.get_cities()

# Convert to DataFrame
df_cities = pd.DataFrame(cities_dict).T   # transpose so each row is one city
df_cities = df_cities[[
    'name',
    'countrycode',
    'latitude',
    'longitude',
    'population'
]]

# Save for reproducibility
df_cities.to_csv('../data/world_cities_updated.csv', index=False)

# Preview
df_cities.head()


,name,countrycode,latitude,longitude,population
3040051,les Escaldes,AD,42.50729,1.53414,15853
3041563,Andorra la Vella,AD,42.50779,1.52109,20430
290594,Umm Al Quwain City,AE,25.56473,55.55517,62747
291074,Ras Al Khaimah City,AE,25.78953,55.9432,351943
291580,Zayed City,AE,23.65416,53.70522,63482
